# 0. Import libraries

In [2]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
import json

import warnings
warnings.filterwarnings("ignore")

In [3]:
from spamint import sprout_plus
from spamint import preprocess as pp

# 1. Load files

In [4]:
inputDir = './tutorial/demo/'
outDir = f'{inputDir}/demo_results/'
sc_exp = pd.read_csv(f'{inputDir}/SC_exp.tsv',sep = '\t',header=0,index_col=0)
sc_meta = pd.read_csv(f'{inputDir}/SC_meta.tsv',sep = '\t',header=0,index_col=0)
st_exp = pd.read_csv(f'{inputDir}/ST_exp.tsv',sep = '\t',header=0,index_col=0)
st_coord = pd.read_csv(f'{inputDir}/ST_coord.tsv',sep = '\t',header=0,index_col=0)
st_decon = pd.read_csv(f'{inputDir}/ST_decon.tsv',sep = '\t',header=0,index_col=0)
# TODO run smurf for demo
sc_smurf = sc_exp + np.random.rand(sc_exp.shape[0],sc_exp.shape[1])*10
lr_df = pd.read_csv('./LR/mouse_LR_pairs.txt',sep='\t',header=None)

# 2.Run sprout

In [5]:
species = 'Mouse'
st_tp = 'st'

In [6]:
st_coord.index = ['spot_' + str(x) for x in st_coord.index]
st_exp.index = ['spot_' + str(x) for x in st_exp.index]
st_decon.index = ['spot_' + str(x) for x in st_decon.index]
sc_exp.index = ['cell_' + str(x) for x in sc_exp.index]
sc_meta.index = ['cell_' + str(x) for x in sc_meta.index]
sc_smurf.index = ['cell_' + str(x) for x in sc_smurf.index]

# Parameters

In [7]:
SUM = 1e4
alpha, beta, gamma, delta, eta = [1, 0.001, 0.001, 0.1, 0.0005]
if st_tp == 'slide-seq':
    num_per_spot = 1
    repeat_penalty = 2
else:
    num_per_spot = 10
    repeat_penalty = int((st_exp.shape[0] * num_per_spot/sc_exp.shape[0]) * 10)

max_rep = 20

In [8]:
from importlib import reload
reload(sprout_plus)

<module 'spamint.sprout_plus' from '/home/notify/Documents/2406-work/SpaMint/spamint/sprout_plus.py'>

# Preprocess

In [9]:
if st_exp.shape[1]<1e4:
    # merfish data, only has 200~500 genes
    sc_adata, st_adata, sc_ref, lr_df = pp.prep_all_adata_merfish(sc_exp = sc_exp, st_exp = st_exp, sc_distribution = sc_smurf, 
                                                        sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)
else:
    sc_adata, st_adata, sc_ref, lr_df = pp.prep_all_adata(sc_exp = sc_exp, st_exp = st_exp, sc_distribution = sc_smurf, 
                                                            sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)

Data clean and scale are done! Single-cell data has 882 genes, spatial data has 882 genes.


In [10]:
obj_spex = sprout_plus.spaMint(save_path = outDir, st_adata = st_adata, weight = st_decon, 
                 sc_ref = sc_ref, sc_adata = sc_adata, cell_type_key = 'celltype', lr_df = lr_df, 
                 st_tp = st_tp)
obj_spex.prep()

sc_ref and sc_adata has different genes, both data are subset to 882 genes.
Parameters checked!
Getting svg genes
By setting k as 4, each spot has average 3.9788359788359786 neighbors.


# cell selection

In [11]:
sc_agg_meta = obj_spex.select_cells(use_sc_orig = True, p = 0, mean_num_per_spot = num_per_spot, mode = 'strict', max_rep = 1,
                                    repeat_penalty = repeat_penalty)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)

	 Estimating the cell number in each spot by the deconvolution result.
	 SpexMod selects 882 feature genes.
	 Init solution: max - 1.0000,     mean - 0.9616,     min - 0.6215
	 Swap selection start...
	 Swapped solution: max - 1.00,     mean - 0.99,     min - 0.60


if p != 0, extremely time consuming

In [12]:
# change p to 0.1, use different code to select cells
#sc_agg_meta = obj_spex.select_cells(use_sc_orig = True, p = 0.1, mean_num_per_spot = num_per_spot, mode = 'strict', max_rep = 1, 
#                                    repeat_penalty = repeat_penalty)
#sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)

	 Estimating the cell number in each spot by the deconvolution result.
	 SpexMod selects 882 feature genes.
	 Init solution: max - 1.0000,     mean - 0.9613,     min - 0.6215
	 Swap selection start...
	 Swapped solution: max - 1.00,     mean - 0.98,     min - 0.60


In [13]:
alter_sc_exp, sc_agg_meta = obj_spex.gradient_descent(
                alpha, beta, gamma, delta, eta, 
                init_sc_embed = False,
                iteration = max_rep, k = 2, W_HVG = 2,
                left_range = 1, right_range = 2, steps = 1, dim = 2)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)
with open(f'{outDir}/sc_knn.json', 'w') as fp:
    json.dump(obj_spex.sc_knn, fp)
# utils.save_object(obj_spex, f'{outDir}/obj_spex.pkl')


Running v12 now...
Init sc_coord by affinity embedding...
Avearge shape correlation is: 0.9768658841593747
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth term calculation done!
Hyperparameters adjusted.
-----Start iteration 0 -----
Avearge shape correlation is: 0.9768658841593747
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth term calculation done!
---0 self.loss4 260.2295080455443 self.GAMMA 0.0031624709994505797 self.GAMMA*self.loss4 0.8229682723953252
---In iteration 0, the loss is:loss1:822.96827,loss2:1914.20351,loss3:1028.00694,loss4:260.22951,loss5:2385.90552.
---In iteration 0, the loss is:loss1:822.96827,loss2:822.96827,loss3:0.82297,loss4:0.82297,loss5:82.29683.
The total loss after iteration 0 is 1729.87931.
-----Start iteration 1 -----
Avearge shape correlation is: 0.9764766821967036
First-term calculation done!
Second-term calculation done!
Third term calculation done!
Fourth te